# Chapter 23: Unit Testing for Quality Code

As your codebase grows, manually testing every feature becomes impossible. Changes that work today might break something else tomorrow. **Unit testing** is the practice of writing automated tests that verify the behavior of individual units of code (typically methods or classes) in isolation. Unit tests give you confidence that your code works correctly and continues to work as you refactor or add new features.

In this chapter, you'll learn:

- The importance of unit testing and the testing pyramid.
- How to set up a unit testing project using **xUnit** (the most popular framework) and **NUnit**.
- The structure of a unit test: **Arrange‑Act‑Assert**.
- Using **assertions** to verify expected outcomes.
- **Mocking** dependencies with **Moq** to isolate the code under test.
- Testing methods that throw exceptions.
- **Code coverage** concepts and tools.
- Best practices for writing maintainable tests.
- A practical example: testing the `OrderService` from Chapter 21.
- Integration testing vs. unit testing.

By the end, you'll be able to write comprehensive unit tests that protect your code from regressions and document its behavior.

---

## 23.1 Why Unit Testing?

Unit testing provides numerous benefits:

- **Bug prevention** – catch errors early in the development cycle.
- **Confidence in refactoring** – change code without fear of breaking existing functionality.
- **Documentation** – tests serve as executable specifications of how code should behave.
- **Design feedback** – if code is hard to test, it's often a sign of poor design (tight coupling, too many responsibilities).
- **Faster debugging** – when a test fails, it's usually clear what went wrong.

The **testing pyramid** suggests that most tests should be unit tests (fast and cheap), with fewer integration tests (testing multiple components together) and even fewer end‑to‑end tests (testing the whole system).

---

## 23.2 Setting Up a Unit Testing Project

Unit tests are typically placed in a separate project, following naming conventions like `MyProject.Tests`. You can create one using the .NET CLI:

```bash
dotnet new xunit -n MyProject.Tests
```

This creates a project with references to the xUnit framework. Other popular frameworks are NUnit and MSTest. We'll focus on xUnit because it's the default for modern .NET.

Add a reference to your main project:

```bash
dotnet add reference ../MyProject/MyProject.csproj
```

If you need mocking, also add Moq:

```bash
dotnet add package Moq
```

---

## 23.3 Anatomy of a Unit Test

A unit test typically follows the **Arrange‑Act‑Assert** pattern:

- **Arrange** – set up the objects and conditions for the test.
- **Act** – execute the method or unit under test.
- **Assert** – verify that the result matches expectations.

Here's a simple test for a `Calculator` class:

```csharp
public class Calculator
{
    public int Add(int a, int b) => a + b;
}

public class CalculatorTests
{
    [Fact]
    public void Add_ShouldReturnSumOfTwoNumbers()
    {
        // Arrange
        var calculator = new Calculator();
        int a = 5;
        int b = 3;

        // Act
        int result = calculator.Add(a, b);

        // Assert
        Assert.Equal(8, result);
    }
}
```

`[Fact]` marks a test method in xUnit. NUnit uses `[Test]`.

---

## 23.4 Assertions

Assertions are provided by the testing framework. xUnit includes many `Assert` methods:

- `Assert.Equal(expected, actual)` – checks equality.
- `Assert.NotEqual(expected, actual)`
- `Assert.True(condition)`, `Assert.False(condition)`
- `Assert.Null(object)`, `Assert.NotNull(object)`
- `Assert.Throws<Exception>(() => ...)` – verifies that a specific exception is thrown.
- `Assert.Contains(expectedSubstring, actualString)`
- `Assert.Collection` – for verifying collections with multiple assertions.

For example, testing exception throwing:

```csharp
[Fact]
public void Deposit_NegativeAmount_ThrowsArgumentException()
{
    var account = new BankAccount();
    Assert.Throws<ArgumentException>(() => account.Deposit(-100));
}
```

---

## 23.5 Mocking with Moq

When a class depends on external components (e.g., databases, web services, file systems), you don't want to use the real implementations in unit tests. They would be slow, fragile, and test more than one unit. Instead, you create **mock objects** that simulate the behavior of dependencies.

**Moq** is the most popular mocking library. It allows you to create mock implementations of interfaces or classes (with virtual methods).

Suppose we have:

```csharp
public interface IEmailSender
{
    void Send(string to, string subject);
}

public class OrderService
{
    private readonly IEmailSender _emailSender;
    public OrderService(IEmailSender emailSender) => _emailSender = emailSender;

    public void PlaceOrder(Order order)
    {
        // ... business logic
        _emailSender.Send(order.CustomerEmail, "Order Confirmed");
    }
}
```

To test `OrderService` without actually sending emails:

```csharp
using Moq;

public class OrderServiceTests
{
    [Fact]
    public void PlaceOrder_ShouldSendConfirmationEmail()
    {
        // Arrange
        var mockSender = new Mock<IEmailSender>();
        var service = new OrderService(mockSender.Object);
        var order = new Order { CustomerEmail = "customer@example.com" };

        // Act
        service.PlaceOrder(order);

        // Assert
        mockSender.Verify(s => s.Send("customer@example.com", "Order Confirmed"), Times.Once);
    }
}
```

- `Mock<IEmailSender>` creates a mock.
- `mockSender.Object` is the fake object that implements the interface.
- `Verify` checks that the method was called with the expected arguments exactly once.

You can also set up mock behavior:

```csharp
mockRepo.Setup(r => r.Get(It.IsAny<int>())).Returns(new Product { Id = 1, Name = "Test" });
```

---

## 23.6 Testing Async Methods

Testing async methods is straightforward – the test method must be `async` and return `Task`. Use `await` to call the method under test.

```csharp
public class DataService
{
    public async Task<string> FetchDataAsync() => await Task.FromResult("data");
}

[Fact]
public async Task FetchDataAsync_ShouldReturnData()
{
    var service = new DataService();
    string result = await service.FetchDataAsync();
    Assert.Equal("data", result);
}
```

For mocking async methods, you can use `ReturnsAsync`:

```csharp
mockRepo.Setup(r => r.GetAsync(1)).ReturnsAsync(new Product());
```

---

## 23.7 Code Coverage

Code coverage measures how much of your production code is exercised by tests. While 100% coverage is not always necessary (and can be misleading), it's a useful metric to identify untested paths.

Tools like **Coverlet** (integrated with xUnit) and **dotnet test --collect:"XPlat Code Coverage"** can generate coverage reports. Visual Studio Enterprise has built‑in coverage tools, and there are third‑party services like Coveralls or Codecov.

Aiming for high coverage on critical business logic is a good practice, but don't obsess over the percentage; focus on testing behavior, not lines.

---

## 23.8 Best Practices for Unit Testing

### 1. Test One Thing per Test

Each test should verify a single behavior. If a test fails, you should know exactly what broke.

### 2. Use Descriptive Names

Test names should describe the scenario and expected outcome, e.g., `PlaceOrder_WithValidOrder_ShouldSaveToDatabase`.

### 3. Keep Tests Independent

Tests should not depend on each other or share state. Each test should be able to run alone.

### 4. Avoid Logic in Tests

Tests should be simple sequences of calls and assertions. Avoid `if` statements, loops, or complex logic that could itself be buggy.

### 5. Use Mocks Only for External Dependencies

Mock interfaces or abstract classes that represent boundaries (I/O, services). For plain domain objects, use real instances.

### 6. Don't Mock What You Don't Own

Avoid mocking types you don't control (e.g., `DateTime.Now`). Instead, wrap them in a small interface you control.

### 7. Write Tests Before Code (TDD)

Test‑Driven Development (TDD) is a discipline where you write a failing test first, then implement just enough code to make it pass, then refactor. This ensures testability and high coverage.

### 8. Keep Tests Fast

Slow tests discourage frequent running. Aim for unit tests that execute in milliseconds.

### 9. Run Tests Frequently

Integrate tests into your build process (CI/CD). Run them locally before pushing.

### 10. Treat Test Code with the Same Care as Production Code

Tests need maintenance too. Keep them clean and readable.

---

## 23.9 Practical Example: Testing OrderService

Let's write comprehensive tests for the `OrderService` from Chapter 21, which depends on `IOrderRepository`, `IEmailSender`, and `ILogger`.

We'll use xUnit and Moq.

```csharp
using Moq;
using Xunit;

public class OrderServiceTests
{
    private readonly Mock<IOrderRepository> _mockRepo;
    private readonly Mock<IEmailSender> _mockEmail;
    private readonly Mock<ILogger> _mockLogger;
    private readonly OrderService _service;

    public OrderServiceTests()
    {
        _mockRepo = new Mock<IOrderRepository>();
        _mockEmail = new Mock<IEmailSender>();
        _mockLogger = new Mock<ILogger>();
        _service = new OrderService(_mockRepo.Object, _mockEmail.Object, _mockLogger.Object);
    }

    [Fact]
    public void PlaceOrder_ShouldSaveOrderAndSendEmail()
    {
        // Arrange
        var order = new Order { CustomerName = "Alice", Amount = 100 };

        // Act
        _service.PlaceOrder(order.CustomerName, order.Amount);

        // Assert
        _mockRepo.Verify(r => r.Save(It.Is<Order>(o => o.CustomerName == "Alice" && o.Amount == 100)), Times.Once);
        _mockEmail.Verify(e => e.Send("orders@company.com", It.Is<string>(s => s.Contains("Alice"))), Times.Once);
        _mockLogger.Verify(l => l.Log(It.Is<string>(s => s.Contains("placed"))), Times.Once);
    }

    [Fact]
    public void PlaceOrder_ShouldAssignNewId()
    {
        // Arrange
        Order savedOrder = null;
        _mockRepo.Setup(r => r.Save(It.IsAny<Order>()))
                 .Callback<Order>(o => savedOrder = o);

        // Act
        _service.PlaceOrder("Bob", 200);

        // Assert
        Assert.NotNull(savedOrder);
        Assert.True(savedOrder.Id > 0); // assuming repository assigns ID
        Assert.Equal("Bob", savedOrder.CustomerName);
        Assert.Equal(200, savedOrder.Amount);
    }

    [Fact]
    public void GetOrder_ExistingId_ReturnsOrder()
    {
        // Arrange
        var expected = new Order { Id = 1, CustomerName = "Alice", Amount = 100 };
        _mockRepo.Setup(r => r.Get(1)).Returns(expected);

        // Act
        var result = _service.GetOrder(1);

        // Assert
        Assert.Same(expected, result);
    }

    [Fact]
    public void GetOrder_NonExistingId_ReturnsNull()
    {
        // Arrange
        _mockRepo.Setup(r => r.Get(99)).Returns((Order)null);

        // Act
        var result = _service.GetOrder(99);

        // Assert
        Assert.Null(result);
    }
}
```

**Explanation:**

- The constructor sets up the mocks and creates the service once per test.
- Each test focuses on one aspect.
- We use `Verify` to check that methods were called.
- For `PlaceOrder_ShouldAssignNewId`, we capture the saved order to inspect it.
- For `GetOrder` tests, we set up the mock repository to return specific values.

---

## 23.10 Integration Testing vs. Unit Testing

While unit tests isolate a single class, **integration tests** verify that multiple components work together correctly. They might use a real database, a test web server, or actual file I/O.

In ASP.NET Core, you can write integration tests using `WebApplicationFactory<T>` to spin up an in‑memory test server.

Integration tests are slower and more brittle, so they should be used sparingly, primarily for critical paths and to catch issues that unit tests miss.

A simple integration test for our Notes API might look like:

```csharp
public class NotesApiTests : IClassFixture<WebApplicationFactory<Program>>
{
    private readonly HttpClient _client;

    public NotesApiTests(WebApplicationFactory<Program> factory)
    {
        _client = factory.CreateClient();
    }

    [Fact]
    public async Task GetNotes_ReturnsSuccessStatusCode()
    {
        var response = await _client.GetAsync("/api/notes");
        response.EnsureSuccessStatusCode();
    }
}
```

This tests the entire stack (routing, controllers, EF Core) but uses a real HTTP client. You can also replace the database with an in‑memory SQLite database for isolation.

---

## 23.11 Chapter Summary

In this chapter, you've learned how to write unit tests that ensure your code behaves as expected:

- **Unit testing** verifies individual units of code in isolation.
- **xUnit** is the recommended testing framework.
- Tests follow **Arrange‑Act‑Assert**.
- **Assertions** verify outcomes.
- **Moq** creates mocks to replace real dependencies.
- **Code coverage** helps identify untested code.
- **Best practices** keep tests maintainable and effective.
- A practical example tested the `OrderService` thoroughly.

Unit testing is an essential skill for professional developers. It not only catches bugs but also guides design and documents behavior.

In the next chapter, **Concurrency and Parallelism**, you'll explore how to write multi‑threaded code safely and efficiently, handling tasks that run simultaneously, avoiding race conditions and deadlocks, and using the Task Parallel Library (TPL).

**Exercises:**

1. Write unit tests for the `BankAccount` class from Chapter 11. Test deposit, withdrawal, and that exceptions are thrown for invalid amounts.
2. For the Notes API from Chapter 22, write unit tests for the `NotesController` using mocks for `AppDbContext` (you'll need to mock `DbSet<T>` – this is more advanced; you can use an in‑memory database or a mocking library like `EntityFrameworkCore.Testing`).
3. Practice TDD: Write a test for a new feature (e.g., a method that calculates discounts) before implementing it.
4. Use Moq to verify that a method was **not** called under certain conditions (use `Times.Never`).
5. Set up code coverage in your project using Coverlet and generate a report.

Now, get ready to tackle concurrency in Chapter 24!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='22. building_web_applications_with_aspnet_core.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../5. advanced_architecture_and_industry_practices/24. concurrency_and_parallelism.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
